# quickProduceReadHitsTable.ipynb
## Marcus Viscardi,    April 24, 2024

I am just taking the first few parts of DESeq2_fromGeneCountsDF.ipynb and putting them here for easier access.

In [ ]:
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
import pandas as pd
from pathlib import Path

import seaborn as sea
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

import nanoporePipelineCommon as npCommon

from icecream import ic
from datetime import datetime

def __time_formatter__():
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return f"ic: {now} | > "
ic.configureOutput(prefix=__time_formatter__)


ic("Imports done.")
working_dir = Path.cwd()
_ = ic(working_dir)

In [ ]:
gene_id_gene_name_df = npCommon.gene_names_to_gene_ids()
gene_id_gene_name_df.head()

In [ ]:
obj_dict = {}
libs_to_run = [
    # "oldN2",
    # "oldS6",
    # "newerN2",
    # "newerS6",
    # "newerS5",
    # "thirdN2",
    # "thirdS5",
    # "thirdS6",
    # "polyA",
    "polyA1",
    "polyA2",
    "polyA3",
    "totalRNA1",
    "totalRNA2",
    "totalRNA3",
]
for lib in libs_to_run:
    print(f"\nLoading {lib}...", end="")
    obj_dict[lib] = npCommon.NanoporeRun(run_nickname=lib)
    print(" Done!")

In [ ]:
# compressedOnGenes_dict = {}
# for lib, obj in obj_dict.items():
#     compressedOnGenes_dict[lib] = obj.load_compressedOnGenes()  # Looks like the old N2 library had a read cutoff of 5 while everything else had no cutoff!!
#     # ic(obj)
# read_hits_series_dict = {}
# for lib, df in compressedOnGenes_dict.items():
#     print(f"Pre-cutdown:  {lib} - {df.shape[0]:,} Genes", end=" ")
#     # # TODO: Eventually, I should rerun the compressing for oldN2 without the cutoff!!!
#     # df = df.query("read_hits >= 5")
#     print(f"Post-cutdown: {lib} - {df.shape[0]:,} Genes")
#     # print(df.head())
#     hits_series = df[[
#         # 'gene_id',
#         'read_hits',
#                       ]]# .set_index('gene_id')
#     print(hits_series)
#     hits_series.rename(columns={'read_hits': lib}, inplace=True)
#     read_hits_series_dict[lib] = hits_series
# read_hits_table = pd.concat(read_hits_series_dict.values(), axis=1).fillna(0)
# read_hits_table.to_csv(working_dir / f"read_hits_table_{'-'.join(libs_to_run)}.csv")
# print(f"Saved read_hits_table_{'-'.join(libs_to_run)}.csv to {working_dir}!")
# read_hits_table.head()

# From fresh featureCounts runs

Another way to go about this would be to run FeatureCounts for each of the libraries using their BAM files, then use the resulting gene count tables to make a read hits table. This would be a bit more work, but would also be a bit more accurate. I will try this next.

In [ ]:
import subprocess

regen = True
threads = 32
input_bam_paths = {lib_name: obj.bam_path for lib_name, obj in obj_dict.items()}
overall_output_dir = working_dir / "featureCounts_testing"
overall_output_dir.mkdir(exist_ok=True)


for lib, lib_obj in obj_dict.items():
    bam_path = lib_obj.bam_path
    gtf_path = lib_obj.gtf_path
    output_dir = overall_output_dir / lib
    output_dir.mkdir(exist_ok=True)
    
    assigned_read_output_file = output_dir / (str(Path(bam_path).name) + ".featureCounts")
    gene_count_output_file = output_dir / f"{npCommon.get_dt(for_file=True)}_{lib}_featureCounts"
    if regen or not gene_count_output_file.exists():
        featCounts_call = (f"featureCounts -L -T {threads} -R CORE -a {gtf_path} "
                           f"-o {output_dir}/{npCommon.get_dt(for_file=True)}_{lib}_featureCounts "
                           f"--largestOverlap -s 1 "
                           f"{bam_path}")
        # TODO: Turn back on
        subprocess.run(featCounts_call, shell=True)
    else:
        print(f"Already ran {lib}! w/ input file {bam_path}, gtf file {gtf_path} and output file {gene_count_output_file}!")
    
    # These would be names for the pure featureCounts output: names=["GeneID", "Chr", "Start", "End", "Strand", "Length", lib]
    featCounts_df = pd.read_csv(gene_count_output_file, sep="\t", skiprows=2, names=["GeneID", "Chr", "Start", "End", "Strand", "Length", lib])
    print(featCounts_df.head())
    break

In [ ]:
genes_to_print = [
    'WBGene00023068',
    'WBGene00023067',
]

featCounts_df.query(f"index in @genes_to_print")

In [ ]:
regen = False
gtf_path = obj_dict['polyA1'].gtf_path
bam_paths_dict = {lib: obj.bam_path for lib, obj in obj_dict.items()}
bam_paths = [str(lib_bam_path) for lib, lib_bam_path in bam_paths_dict.items()]
libs = list(obj_dict.keys())
output_dir = overall_output_dir / "allLibs_fractional"
output_dir.mkdir(exist_ok=True)

gene_count_alllibs_output_file = output_dir / f"{npCommon.get_dt(for_file=True)}_{'-'.join(libs)}_featureCounts"
if regen or not gene_count_alllibs_output_file.exists():
    featCounts_call = (f"featureCounts -L -T {threads} -R CORE -a {gtf_path} "
                       f"-o {gene_count_alllibs_output_file} "
                       # f"-O "  # this will count all features that a read overlaps, instead of tossing it!
                       # f"--fraction "
                       f"--largestOverlap -s 1 "
                       f"{' '.join(bam_paths)}")
    subprocess.run(featCounts_call, shell=True)
else:
    print(f"Already ran {'-'.join(libs)}! w/ input files {bam_paths}, gtf file {gtf_path} and output file {gene_count_alllibs_output_file}!")

In [ ]:
rev_bam_paths_dict = {str(v): k for k, v in bam_paths_dict.items()}
print(rev_bam_paths_dict)

featCounts_alllibs_df = pd.read_csv(gene_count_alllibs_output_file,
                                    sep="\t",
                                    skiprows=1,
                                    # names=["GeneID", "Chr", "Start", "End", "Strand", "Length"] + libs,
                                    )
featCounts_alllibs_df.rename(columns={"Geneid": "Gene_ID"}, inplace=True)
featCounts_alllibs_df.rename(columns=rev_bam_paths_dict, inplace=True)
featCounts_alllibs_df.set_index("Gene_ID", inplace=True)
featCounts_alllibs_simple_df = featCounts_alllibs_df[libs].copy()
featCounts_alllibs_simple_df["sum"] = featCounts_alllibs_simple_df.sum(axis=1)
featCounts_alllibs_simple_df.sort_values("sum", ascending=False, inplace=True)
featCounts_alllibs_simple_df.head(25)

In [ ]:
genes_to_print = [
    'WBGene00023068',
    'WBGene00023067',
]

featCounts_alllibs_simple_df.query(f"index in @genes_to_print")


In [ ]:
featCounts_alllibs_simple_df = featCounts_alllibs_simple_df[['polyA1', 'polyA2', 'polyA3', 'totalRNA2', 'totalRNA3']].copy()
for col in featCounts_alllibs_simple_df.columns:
    print(f"{col} total assigned reads: {featCounts_alllibs_simple_df[col].sum():,}")
featCounts_alllibs_simple_df.head(50)

In [ ]:
featCounts_alllibs_simple_df.to_csv(working_dir / f"featureCounts_readCounts_{'-'.join(featCounts_alllibs_simple_df.columns)}.csv")

In [ ]:
genes_to_print = [
    'WBGene00023068',
    'WBGene00023067',
    'WBGene00004446',
    'WBGene00004419',
    'WBGene00004451',
    'WBGene00004432',
]

featCounts_alllibs_simple_df.query(f"index in @genes_to_print")